In [1]:
import numpy as np
# from gensim.models import Word2Vec
from itertools import chain
from collections import Counter
import os
from tqdm import trange
import multiprocessing
import string, re
import shutil

In [2]:
w2v = dict()
with open('watson partition/Quora_question_pair_partition/wordvec.txt') as f:
    for line in f:
        temp = line.split()
        try:
            w2v[' '.join(temp[0:len(temp)-300])] = list(map(float, temp[-300:]))
        except Exception as e:
            print(e, line)
            continue
len(w2v)

106685

In [3]:
with open('watson partition/Quora_question_pair_partition/test.tsv', 'r') as f:
    lines = f.readlines()
    lines = [l.split('\t')[:-1] for l in lines]
l = len(lines)

def clean(sent):
    sent = str(sent).lower().split()
#     sent = ''.join('#' if i.isdigit() else i for i in sent if i not in string.punctuation)
#     sent = re.sub('#+', '#', sent).split()
    sent = [k for k in sent if k in w2v]
    return sent
data = []
for i in range(l):
    try:
        data.append([clean(lines[i][1]), clean(lines[i][2]), lines[i][0]])
    except Exception as e:
        continue
data = [[i[0], i[1], i[2]] for i in data if not (not i[0] or not i[1])]

In [5]:
all_qs = np.concatenate((np.array(data)[:,0], np.array(data)[:,1]), axis=0)
vocab = Counter(chain(*all_qs))
vocab = {k:v for k, v in vocab.items() if v >= 2}
for i in range(l):
    data[i][0] = [x if x in vocab else 'unk' for x in data[i][0]]
    data[i][1] = [x if x in vocab else 'unk' for x in data[i][1]]

In [29]:
m = Word2Vec(np.concatenate([np.array(data)[:, 0], np.array(data)[:, 1]], 0), batch_words=1000, sg=1, negative=64, 
             min_count=2, iter=2, size=200, workers=multiprocessing.cpu_count())

In [16]:
# m.save('embeddings.model')
m = Word2Vec.load('embeddings.model')

In [4]:
# max_length = max([max(len(i), len(j)) for i, j, _ in data])
max_length = 272
print(max_length)
batch_size = 64
dim = 300

272


In [5]:
i2w = dict(enumerate(list(w2v.keys())))
w2i = {v:k for k, v in i2w.items()}
vocab_size = len(i2w)
for i in range(l):
    data[i][0] = [w2i[j] for j in data[i][0]] + [vocab_size] * (max_length -len(data[i][0]))
    data[i][1] = [w2i[j] for j in data[i][1]] + [vocab_size] * (max_length -len(data[i][1]))

In [6]:
q1 = np.array([i[0] for i in data])
q2 = np.array([i[1] for i in data])
y = np.array([i[2] for i in data])
print(q1.shape, q2.shape, y.shape)

((10000, 272), (10000, 272), (10000,))


In [7]:
vec = np.zeros([vocab_size + 1, dim])
for i, j in i2w.items():
    vec[i] = w2v[j]
vec[vocab_size] = [0] * 300

In [8]:
def batch(x, y, z, size):
    idx = np.random.choice(range(len(x)), size=size, replace=False)
    return x[idx], y[idx], z[idx]

In [8]:
rm -r data

In [9]:
# writing test data
if os.path.exists('data_test'):
    shutil.rmtree('data_test')
os.mkdir('data_test')
for i in range(0, 10000, 64):
    np.savez('data_test/' + str(i), ques1=q1[i:i+64], ques2=q2[i:i+64], label=y[i:i+64])

In [14]:
if not os.path.exists('data'):
    os.mkdir('data')
# idx = np.random.choice(q1.shape[0], 100000, replace=False)
# q1_small, q2_small, y_small = q1[idx], q2[idx], y[idx]
# print(Counter(y))
for i in trange(6000):
    a, b, c = batch(q1, q2, y, 64)
    np.savez('data/'+str(i)+'.npz', ques1=a, ques2=b, label=c)

  0%|          | 4/6000 [00:00<03:08, 31.78it/s]

Counter({'0': 245042, '1': 139306})


100%|██████████| 6000/6000 [02:44<00:00, 36.46it/s]


In [13]:
np.savez('data/embed.npz', embed=vec)

In [13]:
384348 / 64

6005